# ImageNet-10 preparation

In [ ]:
import data_split
import data_converter
import vgg19bn

In [ ]:
ds = data_split.DataSplit()
ds.save_split('data')

In [ ]:
%%time
dc = data_converter.DataConverter(vgg19bn.VGG19bn())
dc.convert_split()

# Mapper

In [7]:
import numpy as np
import mapper
import my_umap
from sklearn.cluster import DBSCAN

In [15]:
loaded = np.load('data/in10_split_converted.npz', allow_pickle=True)
x_train = loaded['x_train']
y_train = loaded['y_train']
x_train.shape, y_train.shape

((10000, 5096), (10000,))

In [16]:
experiment_name = 'comp12_int10_umap12rs69_dbscan50_uniform'

In [17]:
%%time

m = mapper.Mapper()
m.fit(x_train, projector=my_umap.MyUMAP(n_components=12, random_state=69), 
      clusterer=DBSCAN(eps=50, min_samples=1), n_components=12, n_intervals=10, 
      experiment_name=experiment_name)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s

[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.0s
[Pipeline] ......... (step 1 of 2) Processing projector, total=   0.4s
[Pipeline] .............. (step 2 of 2) Processing proj, total=   0.0s
[Pipeline] ....... (step 2 of 3) Processing filter_func, total=   0.4s
[Pipeline] ......... (step 1 of 2) Processing projector, total=   0.4s
[Pipeline] .............. (step 2 of 2) Processing proj, total=   0.0s
[Pipeline] ....... (step 2 of 3) Processing filter_func, total=   0.4s
[Pipeline] ......... (step 1 of 2) Processing projector, total=   0.4s[Pipeline] ......... (step 1 of 2) Processing projector, total=   0.5s

[Pipeline] .............. (step 2 of 2) Processing proj, total=   0.0s
[Pipeline] ....... (step 2 of 3) Processing filter_func, total=   0.5s
[Pipel

[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed:  1.4min finished


CPU times: user 5min 43s, sys: 1min 5s, total: 6min 48s
Wall time: 2min 18s


# Representations

In [18]:
import mapper
import numpy as np
import pickle

In [19]:
loaded = np.load('data/in10_split_converted.npz', allow_pickle=True)
x_train = loaded['x_train']
x_test_none = loaded['x_test_none']
x_test_gaussian = loaded['x_test_gaussian']
y_train = loaded['y_train']
y_test = loaded['y_test']

mapper_data = pickle.load(open('experiments/{}'.format(experiment_name), 'rb'))
latent_space, graphs, covers = mapper_data[0], mapper_data[1], mapper_data[2]
k = 5
x_train.shape, x_test_none.shape, x_test_gaussian.shape, y_train.shape, y_test.shape

((10000, 5096), (3000, 5096), (3000, 5096), (10000,), (3000,))

In [20]:
%%time

m = mapper.Mapper()
m.get_representations(x_train, x_test_none, x_test_gaussian, y_train, y_test,
                      k, latent_space, graphs, covers, experiment_name + '_k{}'.format(k))

[wknn]: 100%|██████████| 3000/3000 [02:14<00:00, 22.23it/s]


CPU times: user 8min 13s, sys: 10min 58s, total: 19min 11s
Wall time: 5min 53s


In [21]:
loaded = np.load('experiments/{}_k{}.npz'.format(experiment_name, k), allow_pickle=True)
x_train = loaded['x_train']
x_test_none = loaded['x_test_none']
x_test_gaussian = loaded['x_test_gaussian']
y_train = loaded['y_train']
y_test = loaded['y_test']
x_train.shape, x_test_none.shape, x_test_gaussian.shape, y_train.shape, y_test.shape

((10000, 120), (3000, 120), (3000, 120), (10000,), (3000,))